In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Задание:

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

5.* Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [ ]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [ ]:
X

In [ ]:
y

In [ ]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

## def calc_std_feat

In [ ]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [ ]:
X_st

In [ ]:
def calc_logloss(y, y_pred):
    y_pred_new = np.where(y_pred == 1, y_pred - 1e-7, np.where(y_pred == 0, y_pred + 1e+7, y_pred))
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))  
    err = np.sum(err)    
    return err

In [ ]:
#Применение: 

y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

In [ ]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [ ]:
z = np.linspace(-10, 10, 101)

In [ ]:
probabilities = sigmoid(z)
probabilities

In [ ]:
plt.plot(z, probabilities)
plt.show()

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [ ]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [ ]:
def eval_model_new(X, y, alpha=1e-4, min_error_dist=1e-5, outp_log=False):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    iter_num = 0
    error_dist = np.inf
    min_err = np.inf
    errors = []
    outp_index = 10**(-np.log10(min_error_dist)-2) if -np.log10(min_error_dist)-2>=1 else 1
    while error_dist > min_error_dist:    
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        errors.append(err)
        error_dist = min_err - err
        if error_dist > min_error_dist:
            min_err = err
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        iter_num += 1
        if outp_log:
            if iter_num % outp_index == 0:
                print(iter_num, W, err)
    if outp_log:
        print(iter_num, W, err)
    return W, min_err, iter_num


In [ ]:
W = eval_model(X_st, y, iterations=5000, alpha=1e-4)

In [ ]:
W_new = eval_model_new(X_st, y, alpha=1e-4, outp_log=True)

In [ ]:
def get_alpha(X, y, alphas):
    alphas_list=[]
    for alpha in alphas:
        W,err,n_iter = eval_model_new(X, y, alpha=alpha)
        alphas_list.append((alpha,err,n_iter))
    alphas_list.sort(key=lambda x:x[1])
    print(f'best - alpha: {alphas_list[0][0]},\nresults:\nerr: {alphas_list[0][1]},\nn_iter: {alphas_list[0][2]}')
    return alphas_list[0]

In [ ]:
alphas=[
    1, 0.5,0.1,0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001, 0.000005, 0.000001, 0.0000005, 0.0000001
]
get_alpha(X_st, y, alphas)

In [ ]:
alphas=np.arange(1,10)/10
get_alpha(X_st, y, alphas)

In [ ]:
W_new = eval_model_new(X_st, y, alpha=0.6, outp_log=True)

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).



In [ ]:
def calc_pred_proba(w, x):
    z = np.dot(w, x)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

In [ ]:
y_pred_prob = calc_pred_proba(W, X_st)
y_pred_prob

In [ ]:
y_pred_prob = calc_pred_proba(W_new[0], X_st)
y_pred_prob

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [ ]:
def calc_pred(W, X):
    pred_proba = sigmoid(np.dot(W, X))
    y_pred = np.zeros_like(pred_proba)
    prob_lim = 0.5
    for idx, prob in enumerate(pred_proba):
        if prob > prob_lim:
            y_pred[idx] = 1
    return y_pred

In [ ]:
y_pred = calc_pred(W, X_st)
y_pred

In [ ]:
y_pred = calc_pred(W_new[0], X_st)
y_pred

In [ ]:
y

5*. Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

$$accuracy(a,x) = \frac{1}{l} \sum^{l}_{i=1}[a(x_{i})=y_{i}]$$

In [ ]:
def accuracy(y, y_pred):
    l = len(y)
    TP_TN = 0
    for i in range(l):
        TP_TN += int(y[i]==y_pred[i])
    result = TP_TN / l
    return result

In [ ]:
accuracy_score(y, y_pred), accuracy(y, y_pred)

In [ ]:
def mk_confusion_matrix(y, y_pred):
    n_classes=len(np.unique(y))
    l=len(y)
    conf_matr=np.zeros((n_classes,n_classes), dtype='int')
    for i in range(l):
        conf_matr[int(y[i])][int(y_pred[i])] += 1
    return conf_matr  

In [ ]:
confusion_matrix(y, y_pred), mk_confusion_matrix(y, y_pred)

In [ ]:
def mk_precision(y, y_pred):
    TN, FP, FN, TP = mk_confusion_matrix(y, y_pred).ravel()
    return TP/(TP+FP)

In [ ]:
precision_score(y, y_pred), mk_precision(y, y_pred)

In [ ]:
def mk_recall(y, y_pred):
    TN, FP, FN, TP = mk_confusion_matrix(y, y_pred).ravel()
    return TP/(TP+FN)

In [ ]:
recall_score(y, y_pred), mk_recall(y, y_pred)

$$F_{\beta} = (1 + \beta^{2}) \frac{precision \cdot recall}{\beta^{2} \cdot precision + recall}$$

In [ ]:
def mk_f_score(y, y_pred, beta=1):
    precision = mk_precision(y, y_pred)
    recall = mk_recall(y, y_pred)
    f_score = (1 + beta**2) * (precision * recall)/(beta**2 * precision + recall)
    return f_score

In [ ]:
f1_score(y, y_pred), mk_f_score(y, y_pred)